In [ ]:
import os
import holidays
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
%matplotlib inline
import tensorflow as tf
random.seed(44)
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import lightgbm
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, make_scorer, mean_absolute_error
from sklearn.metrics import mean_squared_error as mse
from sklearn.feature_selection import mutual_info_classif
from sklearn.linear_model import Lasso
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler

In [ ]:
# Set the aesthetic style of the plots    
sns.set()
sns.set_style("whitegrid")
sns.set_style('ticks')
sns.color_palette("Paired")

In [ ]:
#Reading file and date indeing
df = pd.read_csv('../Final Data/Agreegated_Data.csv', parse_dates=['date'], index_col= 'date')

In [ ]:
WINDOW_SIZE = 7

# Feature Engineering

### A ) Adding Time-related Features & Holidays in Ireland

In [ ]:
df = df.reset_index()

#Create Time-Related Features
df['Month'] = df.date.dt.month
df['Year'] = df.date.dt.year
df['Day'] = df.date.dt.day_of_week
df['Season'] = df['Month'].apply(lambda month_number: (month_number%12 + 3)//3)
df = df.set_index(df.date, drop =True)
df.drop(['date', 'Month'], inplace = True, axis = 1)
df['Holiday'] = pd.Series(df.index).apply(lambda x: holidays.CountryHoliday('Ireland').get(x)).values
df['Holiday'] = df['Holiday'].astype('bool').astype('int')

# We slected cotegoric features for one-hot encoding
df= pd.get_dummies(df, columns=['Holiday', 'Season', 'Day'])
df = df.drop('Year', axis = 1)

### B) Adding Historical Data of Traffic Congestion in Dublin

In [ ]:
# Add windowed columns (Features with Lags)
for i in range(WINDOW_SIZE):
    df[f"Congestion+{i+1}"] = df["congestion"].shift(periods=i+1)
df = df.dropna()

In [ ]:
#Detect missing values.
df.isnull().sum()

In [ ]:
#Period
print ('Period of study %', - (df.index.min()- df.index.max()))
df.index.min(), df.index.max()

In [ ]:
# Evaluation Metric Formulation

def mape(actual, pred): 
    actual, pred = np.array(actual), np.array(pred)
    return np.mean(np.abs((actual - pred) / actual)) * 100

def show_scores(model):
    y_pred = model.predict(X_test)
    scores = {'R-squared': r2_score(y_test,y_pred)*100,
              'MSE':  mean_squared_error(y_test,y_pred),
              'RMSE': np.sqrt(mean_squared_error(y_test,y_pred)),
              'MAE': mean_absolute_error (y_test,y_pred),
              'MAPE': mape (y_test,y_pred)}
    return scores    

In [ ]:
#Visualization of Train & Test Split (Start & End for zooming)
def plot_split (timesteps, values, format = ".", start = 0 , end = None, color =None, label =None):   

    plt.plot(timesteps[start:end], values[start:end],format, color = color, label=label)
    
    plt.ylabel("Traffic congestion (%)")
    plt.xlabel('Date')
    plt.xticks(rotation=90)
    plt.legend(fontsize = 12)

In [ ]:
# Plot for Feature importance
def plot_features (columns, importances, n=43):
    df = (pd.DataFrame({"features": columns,
                      "feature_importances": importances})
          .sort_values("feature_importances", ascending = False)
          .reset_index(drop = True))
    #plot
    fig, ax = plt.subplots(figsize=[7,43])

    ax.barh(df["features"][:n],
           df["feature_importances"][:n], color ='seagreen' )
    ax.set_ylabel ("Features")
    ax.set_xlabel("Features importance RF")
    ax.invert_yaxis()
    
    fig.savefig('rfeatureimportance.pdf',
            format='pdf',
            dpi=1800,
            bbox_inches='tight')

## All Features

In [ ]:
HORIZON = 1 # predict next one day
WINDOW_SIZE = 1 # use the past week street waste data

In [ ]:
# Make features and labels
X = df.dropna().drop("congestion", axis=1)
y = df.dropna()["congestion"]

# Make train and test sets
split_size = int(len(X) * 0.8)
X_train, y_train = X[:split_size], y[:split_size]
X_test, y_test = X[split_size:], y[split_size:]
len(X_train), len(y_train), len(X_test), len(y_test)

In [ ]:
plt.figure(figsize = (7,5))
plot_split(timesteps=X_train.index, values=y_train,  color ="#f58231",  label = 'Train date')
plot_split(timesteps=X_test.index, values=y_test,  color ="#911eb4",  label = 'Test date')

# Feature Selection Experiments

## A) Random Forest Feature Selection

In [ ]:
rf = RandomForestRegressor(n_jobs=-1, random_state=42)
rf.fit(X_train, y_train)

In [ ]:
show_scores(rf)

In [ ]:
%%time
#Hyperparameter
rf_grid = {'n_estimators': np.arange (10,100,10) ,
    'max_depth':[None, 3, 5, 10],
    'min_samples_split':np.arange (2,20, 2),
    'min_samples_leaf':np.arange (1,20, 2),
    'max_features':['auto', "sqrt", 0.5, 1]}

In [ ]:
%%time
#Nested cross validatio to only split based on train and test and not using validation set as data is not enough
tuned_rf =RandomizedSearchCV(RandomForestRegressor(n_jobs=-1,random_state=42),
                             param_distributions= rf_grid,
                             cv = 5,
                             n_iter = 20,
                             scoring='neg_mean_squared_error',
                             verbose= True)
tuned_rf.fit(X_train, y_train)                                             

In [ ]:
#The Best Hyoerparameters
tuned_rf.best_params_

### Best HyperParameters

In [ ]:
%%time
ideal_rf = RandomForestRegressor (n_estimators= 80, min_samples_split= 2,
                                  min_samples_leaf= 1, max_features= 'auto',
                                  max_depth= 5,
                                  n_jobs = -1)
ideal_rf.fit(X_train, y_train)

In [ ]:
show_scores(ideal_rf)

In [ ]:
ideal_rf.feature_importances_

In [ ]:
plot_features(X_train.columns,ideal_rf.feature_importances_ )

In [ ]:
rf_scores = ideal_rf.feature_importances_
rf_scores

Mutual INFO:
Estimate mutual information for a discrete target variable.
Mutual information (MI)between two random variables is a non-negative
value, which measures the dependency between the variables. It is equal
to zero if and only if two random variables are independent, and higher
values mean higher dependency.

In [ ]:
i_scores = mutual_info_classif(X_train,y_train)
feature_names = X.columns

In [ ]:
df_gain=pd.DataFrame({'Mutual Info.':i_scores,'RF Score':rf_scores,'Feature':feature_names})
df_gain.set_index('Feature', inplace = True)
df_gain.sort_values('Mutual Info.', inplace = True, ascending = False)
df_gain

In [ ]:
n = len(df_gain.index)
rr = range(0,n)
fig, ax = plt.subplots(figsize=(7,5))
ax2 = ax.twinx()
ax.bar(df_gain.index, df_gain["RF Score"], label='RF Score',width=.35, color = 'g')

ax2.set_xticks(rr)
ax2.plot(df_gain.index, df_gain["Mutual Info."], label='I-Gain', color = 'navy')

ax.set_xticklabels(list(df_gain.index), rotation = 90)
ax.set_xlabel('Features')
ax.set_ylabel('I-Gain')
ax2.set_ylabel('RF Score')
fig.legend(loc="upper right", bbox_to_anchor=(1,1), bbox_transform=ax.transAxes)
plt.show()

## B) LightGBM Feature Selection

In [ ]:
lg = lightgbm.LGBMRegressor(n_jobs=-1, random_state=42)
lg.fit(X_train, y_train)

In [ ]:
# Grid search CV
parameters = {'max_depth'     : [6,7,8,9,10,11,12,13,14,15],
              'learning_rate' : [0.01, 0.05, 0.1],
              'num_iteration' : [1000, 5000, 10000],
              'n_estimators'  : [100,300,500]
              # Add more parameters here for tuning
              }        
tuned_lgb = RandomizedSearchCV(lg,  param_distributions = parameters, cv = 5, 
                    verbose = 1, n_jobs = -1, refit = True)

In [ ]:
tuned_lgb.fit(X_train, y_train) 

In [ ]:
tuned_lgb.best_params_ 

In [ ]:
%%time
tuned_lgb =  lightgbm.LGBMRegressor(n_estimators = 300,max_depth= 12,
                                    learning_rate= 0.01,
                                    num_iteration= 1000 ,
                                    n_jobs = -1)
tuned_lgb.fit(X_train, y_train)

In [ ]:
show_scores(tuned_lgb)

In [ ]:
tuned_lgb.feature_importances_

In [ ]:
plot_features(X_train.columns,tuned_lgb.feature_importances_ )

## C) Pearson Correlation Feature Selection

In [ ]:
# Using Pearson Correlation : Degree in which variables move together
def Correlation(data):
    mask2 = np.zeros_like(data.corr()) # Making the arrays of zeros
    trinangle_indices = np.triu_indices_from(mask2) # triangle of top arrays
    mask2[trinangle_indices] = True # make top triangle 1 and below zero

    cbar_kws = {"shrink":.8,
       'extend':'max',
       'extendfrac':.2, 
       "drawedges":False,
        'label':'Correlation'}
    #plt.figure (figsize = (35,1))
    fig , ax = plt.subplots(figsize = (7,5))
    sns.heatmap(data.corr(), mask = mask2, annot = True, linewidths= 0.5, fmt = '.1f', cmap = 'YlGnBu',annot_kws={'size':14}, cbar_kws=cbar_kws) #annot: values on it

    #increasing fonts
    plt.xticks (fontsize = 14)
    plt.yticks (fontsize = 14)

    plt.show()

In [ ]:
abs(df.corr())["congestion"][abs(df.corr()["congestion"])> 0.50].drop('congestion').index.tolist()

In [ ]:
df.shape

## D) Lasso Feature Selection

In [ ]:
# Make features and labels
X = df.dropna().drop("congestion", axis=1)
y = df.dropna()["congestion"]

# Make train and test sets
split_size = int(len(X) * 0.8)
X_train, y_train = X[:split_size], y[:split_size]
X_test, y_test = X[split_size:], y[split_size:]
len(X_train), len(y_train), len(X_test), len(y_test)

In [ ]:
# Linear Model Benifits with feature Scaling for LASSO
scaler = StandardScaler()
scaler.fit(X_train.fillna(0))

In [ ]:
sel_ = SelectFromModel(Lasso(alpha=0.005,random_state=0))
sel_.fit(scaler.transform(X_train.fillna(0)), y_train)

Next we will be selecting the columns based on how they affect the p-value. We are the removing the column diagnosis because it is the column we are trying to predict

In [ ]:
# this command lets us to vizualise which features were kept
sel_.get_support()

In [ ]:
# We can now make a list of selected features 
selected_feat = X_train.columns[(sel_.get_support())]

print('Total features:{}'.format((X_train.shape[1])))
print('Selected features:{}'.format(len(selected_feat)))
print('Features with coefficients shrank to zero:{}'.format(np.sum(sel_.estimator_.coef_==0)))

In [ ]:
# The number of features which coefficient was shrank to zero 
np.sum(sel_.estimator_.coef_==0)

In [ ]:
# Identifying the removed features 
removed_feats =X_train.columns[(sel_.estimator_.coef_==0).ravel().tolist()]
removed_feats

In [ ]:
selected_feat

## List of Features in all Experiments

In [ ]:
all_features = ['maxtp', 'mintp', 'rain', 'people_vaccinated', 'total_boosters',
       'congestion', 'C1_School closing', 'C2_Workplace closing',
       'C3_Cancel public events', 'C4_Restrictions on gatherings',
       'C5_Close public transport', 'C6_Stay at home requirements',
       'C7_Restrictions on internal movement',
       'C8_International travel controls', 'Retail and Recreation',
       'Grocery and Pharmacy', 'Parks', 'Transit Stations', 'Workplaces',
       'Residential', 'Daily_Confirmed', 'Wate Qty', 'Congestion+1',
       'Congestion+2', 'Congestion+3', 'Congestion+4', 'Congestion+5',
       'Congestion+6', 'Congestion+7', 'Holiday_0', 'Holiday_1', 'Season_1',
       'Season_2', 'Season_3', 'Season_4', 'Day_0', 'Day_1', 'Day_2', 'Day_3',
       'Day_4', 'Day_5', 'Day_6']
RF_Features = ['Congestion+7',
              'Retail and Recreation',
              'Workplaces',
              'Day_6', 
              'Congestion+1', 
               'Wate Qty',
              'Residential', 
              'Holiday_0',
               'Grocery and Pharmacy',
               'Holiday_1', 
               'Parks',
               'Season_1',
               'Congestion+6',
               'Congestion+3',
               'C1_School closing', 
               'Congestion+3']
lg_features = ['maxtp', 'mintp', 'rain', 'people_vaccinated',
       'congestion', 'C1_School closing',
       'C3_Cancel public events',
       'C5_Close public transport', 'C6_Stay at home requirements',
       'C7_Restrictions on internal movement',
       'C8_International travel controls', 'Retail and Recreation',
       'Grocery and Pharmacy', 'Parks', 'Transit Stations', 'Workplaces',
       'Residential', 'Daily_Confirmed', 'Wate Qty', 'Congestion+1',
       'Congestion+2', 'Congestion+3', 'Congestion+4', 'Congestion+5',
       'Congestion+6', 'Congestion+7','Season_1',
       'Season_2', 'Season_3', 'Season_4', 'Day_0', 'Day_1',
       'Day_4', 'Day_5', 'Day_6'] 
lasso_Features = ['maxtp', 'mintp', 'rain', 'people_vaccinated', 
       'congestion', 'C1_School closing', 'C2_Workplace closing',
       'C3_Cancel public events', 'C4_Restrictions on gatherings',
       'C5_Close public transport', 'C6_Stay at home requirements',
       'C7_Restrictions on internal movement',
       'C8_International travel controls',
       'Grocery and Pharmacy','Transit Stations', 'Workplaces',
       'Residential', 'Daily_Confirmed', 'Wate Qty', 'Congestion+1',
       'Congestion+2','Congestion+4', 'Congestion+5',
       'Congestion+6', 'Congestion+7', 'Holiday_0','Season_1',
       'Season_2', 'Season_3', 'Season_4', 'Day_0', 'Day_1','Day_3',
       'Day_4', 'Day_5', 'Day_6']
pearson_Features=['people_vaccinated','C1_School closing', 
                 'Retail and Recreation', 'Grocery and Pharmacy', 'Transit Stations', 
                 'Congestion+1', 'Congestion+2', 'Congestion+6', 'Congestion+7'] 
len (all_features), len(RF_Features), len(lg_features), len(lasso_Features), len(pearson_Features)